In [3]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score

In [4]:
def read_data(trainfile='MNIST_train.csv', validationfile='MNIST_validation.csv'):
    
    dftrain = pd.read_csv(trainfile)
    dfval = pd.read_csv(validationfile)

    featurecols = list(dftrain.columns)
    featurecols.remove('label')
    featurecols.remove('even')

    targetcol1 = 'label'
    targetcol2 = 'even'

    Xtrain = dftrain[featurecols]
    ytrain = dftrain[targetcol1]
    ytrain2 = dftrain[targetcol2]
    
    Xval = dfval[featurecols]
    yval = dfval[targetcol1]
    yval2 = dfval[targetcol2]

    return (Xtrain, ytrain, Xval, yval, ytrain2, yval2)

In [5]:
Xtrain, ytrain, Xval, yval, ytrain_oe, yval_oe = read_data('MNIST_train.csv', 'MNIST_validation.csv')

In [6]:
class PCA:
    """PCA class that can be fitted once and reused"""
    def __init__(self, variance_threshold=0.95, n_components=None):
        self.variance_threshold = variance_threshold
        self.n_components = n_components
        self.components = None
        self.mean = None
        self.explained_variance_ratio = None
        
    def fit(self, X):
        """Fit PCA on training data only"""
        # Normalize and center data
        X_normalized = np.array(X, dtype=float) / 255.0
        self.mean = np.mean(X_normalized, axis=0)
        X_centered = X_normalized - self.mean
        
        # Compute covariance matrix and eigen decomposition
        evd_matrix = (X_centered.T @ X_centered) / (X_centered.shape[0] - 1)
        eigenvalues, eigenvectors = np.linalg.eigh(evd_matrix)
        
        # Sort by descending eigenvalues
        sorted_idx = np.argsort(eigenvalues)[::-1]
        eigenvalues_sorted = eigenvalues[sorted_idx]
        eigenvectors_sorted = eigenvectors[:, sorted_idx]
        
        # Determine components for target variance
        self.explained_variance_ratio = eigenvalues_sorted / np.sum(eigenvalues_sorted)
        cumulative_variance = np.cumsum(self.explained_variance_ratio)

        if self.n_components is None:
            self.n_components = np.argmax(cumulative_variance >= self.variance_threshold) + 1
        
        # Store components
        self.components = eigenvectors_sorted[:, :self.n_components]
        
        print(f"PCA fitted with {self.n_components} components, explaining {cumulative_variance[self.n_components-1]:.4f} variance")
        return self
        
    def transform(self, X):
        """Transform data using fitted PCA"""
        if self.components is None:
            raise ValueError("PCA must be fitted before transforming")
            
        # Apply same normalization and centering
        X_normalized = np.array(X, dtype=float) / 255.0
        X_centered = X_normalized - self.mean
        
        # Project to PCA space
        return X_centered @ self.components
    
    def fit_transform(self, X):
        """Fit and transform in one step"""
        return self.fit(X).transform(X)

In [7]:
class KNN:
    def __init__(self, k=5, distance_metric='euclidean', weights='uniform'):
        self.k = k
        self.distance_metric = distance_metric
        self.weights = weights
        
    def fit(self, X, y):
        self.X_train = np.asarray(X)
        self.y_train = np.asarray(y)
        self.classes = np.unique(y)
        return self

    def _compute_all_distances(self, X):
        """
        Compute distances between X (n_test × d) and train (n_train × d)
        Fully vectorized
        """
        X = np.asarray(X)

        if self.distance_metric == 'euclidean':
            # (x - y)^2 = x^2 + y^2 - 2xy
            X2 = np.sum(X**2, axis=1).reshape(-1, 1)
            T2 = np.sum(self.X_train**2, axis=1).reshape(1, -1)
            dist = np.sqrt(np.maximum(X2 + T2 - 2 * X @ self.X_train.T, 0))

        elif self.distance_metric == 'manhattan':
            # |x - y|
            # Use broadcasting
            dist = np.sum(np.abs(X[:, None, :] - self.X_train[None, :, :]), axis=2)

        elif self.distance_metric == 'cosine':
            # 1 - cosine similarity
            X_norm = X / (np.linalg.norm(X, axis=1, keepdims=True) + 1e-15)
            T_norm = self.X_train / (np.linalg.norm(self.X_train, axis=1, keepdims=True) + 1e-15)
            dist = 1 - X_norm @ T_norm.T

        else:
            raise ValueError("Unsupported metric")

        return dist

    def predict(self, X):
        X = np.asarray(X)

        # vectorized distance computation
        dist = self._compute_all_distances(X)

        # get k nearest neighbors
        idx = np.argpartition(dist, self.k, axis=1)[:, :self.k]
        neighbors = self.y_train[idx]

        if self.weights == 'uniform':
            # majority vote
            return np.array([Counter(row).most_common(1)[0][0] for row in neighbors])

        else:  # distance-weighted
            kdist = np.take_along_axis(dist, idx, axis=1)
            weights = 1.0 / (kdist + 1e-15)
            preds = []
            for lbls, w in zip(neighbors, weights):
                vote = {}
                for label, weight in zip(lbls, w):
                    vote[label] = vote.get(label, 0) + weight
                preds.append(max(vote.items(), key=lambda a: a[1])[0])
            return np.array(preds)

    def predict_proba(self, X):
        X = np.asarray(X)
        dist = self._compute_all_distances(X)

        idx = np.argpartition(dist, self.k, axis=1)[:, :self.k]
        neighbors = self.y_train[idx]

        probs = np.zeros((len(X), len(self.classes)))

        for i, row in enumerate(neighbors):
            for j, cls in enumerate(self.classes):
                probs[i, j] = np.sum(row == cls) / self.k

        return probs

In [11]:
from sklearn.metrics import f1_score
import numpy as np

def hyperparameter_tuning(Xtrain, ytrain, Xval, yval):
    """
    Hyperparameter tuning for custom PCA + custom KNN using WEIGHTED F1.
    """

    k_values = [1, 3, 5, 7, 9, 11, 15, 21]
    n_components_values = [10, 20, 30, 50, 75, 100, 150, 200]

    distance_metrics = ['euclidean', 'manhattan', 'cosine']
    weights_options = ['uniform', 'distance']

    results = []
    best_f1 = 0
    best_params = {}

    print("\n========== Hyperparameter Tuning (Weighted F1) ==========")

    # ----------------------------------------------------------
    # Phase 1: Tune PCA n_components with fixed k=5
    # ----------------------------------------------------------
    print("\nPhase 1: Selecting best PCA dimension...")
    pca_phase_results = []

    for n_components in n_components_values:

        pca = PCA(n_components=n_components)
        Xtrain_pca = pca.fit_transform(Xtrain)
        Xval_pca = pca.transform(Xval)

        knn = KNN(k=5, distance_metric='euclidean', weights='uniform')
        knn.fit(Xtrain_pca, ytrain)
        ypred = knn.predict(Xval_pca)

        f1 = f1_score(yval, ypred, average='weighted')

        pca_phase_results.append({"n_components": n_components, "f1": f1})

        print(f"PCA={n_components:4d} | F1={f1:.4f}")

    best_pca = max(pca_phase_results, key=lambda x: x["f1"])
    optimal_n_components = best_pca["n_components"]

    print(f"\n>>> Best PCA Components: {optimal_n_components}\n")

    # ----------------------------------------------------------
    # Phase 2: Tune k using best PCA dim
    # ----------------------------------------------------------
    print("\nPhase 2: Tuning k values...")

    pca_fixed = PCA(n_components=optimal_n_components)
    Xtrain_pca_fixed = pca_fixed.fit_transform(Xtrain)
    Xval_pca_fixed = pca_fixed.transform(Xval)

    for k in k_values:

        knn = KNN(k=k, distance_metric='euclidean', weights='uniform')
        knn.fit(Xtrain_pca_fixed, ytrain)
        ypred = knn.predict(Xval_pca_fixed)

        f1 = f1_score(yval, ypred, average='weighted')

        results.append({
            "k": k,
            "n_components": optimal_n_components,
            "distance_metric": "euclidean",
            "weights": "uniform",
            "f1": f1
        })

        print(f"k={k:3d} | F1={f1:.4f}")

        if f1 > best_f1:
            best_f1 = f1
            best_params = {
                "k": k,
                "n_components": optimal_n_components,
                "distance_metric": "euclidean",
                "weights": "uniform"
            }

    # ----------------------------------------------------------
    # Phase 3: Tune distance metric + weights using best k, best PCA dim
    # ----------------------------------------------------------
    print("\nPhase 3: Tuning distance metrics & weights...")

    for dist in distance_metrics:
        for w in weights_options:

            knn = KNN(k=best_params["k"], distance_metric=dist, weights=w)
            knn.fit(Xtrain_pca_fixed, ytrain)
            ypred = knn.predict(Xval_pca_fixed)

            f1 = f1_score(yval, ypred, average='weighted')

            results.append({
                "k": best_params["k"],
                "n_components": optimal_n_components,
                "distance_metric": dist,
                "weights": w,
                "f1": f1
            })

            print(f"k={best_params['k']:3d} | metric={dist:10} | weights={w:8} | F1={f1:.4f}")

            if f1 > best_f1:
                best_f1 = f1
                best_params = {
                    "k": best_params["k"],
                    "n_components": optimal_n_components,
                    "distance_metric": dist,
                    "weights": w
                }

    print("\n========== BEST PARAMETERS ==========")
    print(best_params)
    print(f"Best Weighted F1: {best_f1:.4f}")
    print("=====================================\n")

    return results, best_params, best_f1

In [12]:
results, best_params, best_f1 = hyperparameter_tuning(Xtrain, ytrain, Xval, yval)


========== Hyperparameter Tuning (Weighted F1) ==========

Phase 1: Selecting best PCA dimension...
PCA fitted with 10 components, explaining 0.4869 variance
PCA=  10 | F1=0.9058
PCA fitted with 20 components, explaining 0.6434 variance
PCA=  20 | F1=0.9563
PCA fitted with 30 components, explaining 0.7306 variance
PCA=  30 | F1=0.9583
PCA fitted with 50 components, explaining 0.8254 variance
PCA=  50 | F1=0.9579
PCA fitted with 75 components, explaining 0.8837 variance
PCA=  75 | F1=0.9563
PCA fitted with 100 components, explaining 0.9156 variance
PCA= 100 | F1=0.9543
PCA fitted with 150 components, explaining 0.9492 variance
PCA= 150 | F1=0.9539
PCA fitted with 200 components, explaining 0.9672 variance
PCA= 200 | F1=0.9515

>>> Best PCA Components: 30


Phase 2: Tuning k values...
PCA fitted with 30 components, explaining 0.7306 variance
k=  1 | F1=0.9583
k=  3 | F1=0.9559
k=  5 | F1=0.9583
k=  7 | F1=0.9571
k=  9 | F1=0.9571
k= 11 | F1=0.9543
k= 15 | F1=0.9530
k= 21 | F1=0.9478

Ph